# Gaussian diagonal covariance

Here we figure out the Gaussian diagonal covariance and try to code this ourselves.
Look at
- Gaussian covariance for the $C_{l}$'s; (Dodelson, example 12.1; section 14.2)
- Different noise types
- Expressions for the different types of probes
- Wick's theorem

*Guess:*
1. Gaussian diagonal covariance describes an striclty auto-correlated parameter space. This is expressed in a diagonal covariance matrix, in which the variance in each parameter is Gaussian. The fact that the covariance matrix is diagonal means that any cross-correlations are disregarded. Thus, the parameters panning parameter space are *univariant* in auto-correlations and invariant in cross-correlations.

Look at sources:
- **[Section 2.5: Johnston, H., Chisari, N. E., Joudaki, S., Reischke, R., Stölzner, B., Loureiro, A., ... & Zhang, Y. H. (2024). 6x2pt: Forecasting gains from joint weak lensing and galaxy clustering analyses with spectroscopic-photometric galaxy cross-correlations. arXiv preprint arXiv:2409.17377](https://arxiv.org/abs/2409.17377)**
- **[Later chapters of: Modern Cosmology; S. Dodelson, F. Schmidt](https://books.google.se/books?hl=nl&lr=&id=TpUmEQAAQBAJ&oi=fnd&pg=PP1&dq=modern+cosmology+dodelson&ots=BBo0fsLhcW&sig=KC-2ksaqhpjJvRv05wjpd2nS8Lg&redir_esc=y#v=onepage&q=modern%20cosmology%20dodelson&f=false)**


In [25]:
# The required imports:

import numpy as np
import pyccl as ccl
import pylab as plt
import math
import glass.observations
%matplotlib inline

## Analytical expression:
We can write down the analytical expression for the Gaussian covariance through:
$Z = Cov\left[\hat{C}^{ij}(\ell)\hat{C}^{mn}(\ell')\right]$, with:
$$Cov\left[\hat{C}^{ij}(\ell)\hat{C}^{mn}(\ell')\right] = \frac{\delta_{\ell\ell'}^{K}}{f_{sky}N_{\ell}}\left(\hat{C}^{im}(\ell)\hat{C}^{jn}(\ell') + \hat{C}^{in}(\ell)\hat{C}^{jm}(\ell')\right)$$

where $4\pi f_{sky} = A_{survey}$ the fraction of the sky considered in the survey, $\delta_{\ell\ell'}^{K}$ is the K-dimensional Dirac delta which ensures that the multipole moments ($\ell$) are independent. In this expression we must note that $\hat{C}^{ij}(\ell)$ is the observed angular power spectrum. This means that it contains both the signal $\left(S^{ij}(\ell)\right)$ and the noise $\left(N^{ij}\right)$:
$$\hat{C}^{ij}(\ell) = S^{ij}(\ell) + N^{ij}$$

More specifically,
$$
\hat{C}^{ij} (\ell) = \begin{cases}
S^{ij} (\ell) + N_{source}^{ij}, & \text{if $i \wedge j \in$ source}.\\
S^{ij} (\ell) + N_{lens}^{ij}, & \text{if $i \wedge j \in$ lens}.\\
S^{ij} (\ell), & \text{else}
\end{cases}
$$

Where the noise differs for the case of galaxy clustering, weak lensing or CMB lensing. Note also that the noise only occurs for the auto-correlations (e.g. the diagonal of the covariance). 

Note: For CMB lensing, we make use of the noise as defined by Planck, stowed away in the file 'cmb_noise_Planck.txt' which we read in and use when needed.

In [26]:
# Now we define a function that gives us the Gaussian coavariance based on the probe, the corresponding noise and surveysurface on the observed sky,
# a redshift distribution, bins and their binwidth.

def Gauss_cov(l, ang_power_spectrum, noise, f_sky, redshift_dist=None, bins=None, cross=False):
    # We calculate the Gaussia covariance based on the true-model tracer. The angular power spectrum that represents LCDM
    cov = np.zeros(shape=(len(ang_power_spectrum), len(ang_power_spectrum)))
    
    # We separate the cases of auto and cross-correlations:
    if cross == False:
        
        # Now we need to take into account the noise acompanying the angular power spectrum
        for i in range(len(l)):
            for j in range(len(l)):

                # For the diagonal elements, we compute auto correlations of the same probes, thus we includer the noise terms. Assume
                if i == j:
                    cov[i, j] = 1/(f_sky * (2 * l[i] + 1)) * (ang_power_spectrum[i] * ang_power_spectrum[i] + ang_power_spectrum[i] * ang_power_spectrum[i] + noise[i] * ang_power_spectrum[i] + ang_power_spectrum[i] * noise[i])
                
                # For the non-diagonal elements:
                else:
                    cov[i, j] = 1/(f_sky * (2 * l[i] + 1)) * (ang_power_spectrum[i] * ang_power_spectrum[i] + ang_power_spectrum[i] * ang_power_spectrum[i])
    
    # Now for the cross-correlations:
    else:
        for i in range(len(l)):
            for j in range(len(l)):
                cov[i, j] = 1/(f_sky * (2 * l[i] + 1)) * (ang_power_spectrum[i] * ang_power_spectrum[j])
    
    return cov

tracer = np.array([1, 2, 56, 7, 9, 2, 7])
noise = np.zeros_like(tracer)
l = np.linspace(10, 2000, len(tracer))

print("A", Gauss_cov(l, tracer, noise, 12, cross=True).shape, "matrix, which contains the following:\n", Gauss_cov(l, tracer, noise, 12, cross=True))

A (7, 7) matrix, which contains the following:
 [[3.96825397e-03 7.93650794e-03 2.22222222e-01 2.77777778e-02
  3.57142857e-02 7.93650794e-03 2.77777778e-02]
 [2.43546030e-04 4.87092060e-04 1.36385777e-02 1.70482221e-03
  2.19191427e-03 4.87092060e-04 1.70482221e-03]
 [3.46277517e-03 6.92555033e-03 1.93915409e-01 2.42394262e-02
  3.11649765e-02 6.92555033e-03 2.42394262e-02]
 [2.90071275e-04 5.80142549e-04 1.62439914e-02 2.03049892e-03
  2.61064147e-03 5.80142549e-04 2.03049892e-03]
 [2.80443724e-04 5.60887449e-04 1.57048486e-02 1.96310607e-03
  2.52399352e-03 5.60887449e-04 1.96310607e-03]
 [4.99350844e-05 9.98701688e-05 2.79636473e-03 3.49545591e-04
  4.49415760e-04 9.98701688e-05 3.49545591e-04]
 [1.45796884e-04 2.91593768e-04 8.16462551e-03 1.02057819e-03
  1.31217196e-03 2.91593768e-04 1.02057819e-03]]


## Generate the angular power spectra
To generate these we first get 

In [29]:
# We define the cosmology:
lcdm_cosmo = ccl.Cosmology(Omega_c=0.27, Omega_b=0.045, h=0.67, A_s=2.1e-9, n_s=0.96)

# Alternate neutrino cosmology:
neutrino_cosmo = ccl.Cosmology(Omega_c=0.27, Omega_b=0.045, h=0.67, A_s=2.1e-9, n_s=0.96, m_nu=0.12, mass_split='normal')
zero_mm_cosmo = ccl.Cosmology(Omega_c=0.27, Omega_b=0.045, h=0.67, A_s=2.1e-9, n_s=0.96, m_nu=[0, 0.06, 0.06])

# The specifics of the angular power spectra:
l = np.linspace(2, 2000, 1000) # multipole moments
z = np.linspace(0, 3, 1000) # redshifts
a = (1./(1+z))[::-1]

# Redshoft distribution:
nz = np.exp(-((z-0.5)/0.05)**2/2)

# Biases:
bias = 0.95/ccl.growth_factor(lcdm_cosmo, a) # bias
m_bias = np.ones_like(z) # Magnification bias
i_a = -0.004 * np.ones_like(z) # Intrinsic alignment amplitude

### Binning:
Define the spectroscopic and photometric bins

In [34]:
# We define the photometric and spectra through the following:

# Photometric:
n_bar = 0.3 # Assume a galaxy density and accompanying reshift distribution:
dndz = glass.observations.smail_nz(z, z_mode=0.9, alpha=2., beta=1.5)
dndz *= n_bar
zbins = glass.observations.equal_dens_zbins(z, dndz, nbins=6)
phot_tomo_bins = glass.observations.tomo_nz_gausserr(z, dndz, 0.03, zbins) # Where the bin error is 0.03

# Spectroscopic:
spec_z = (np.heaviside(z - 0.9, 1) - np.heaviside(z - 0.8, 1)) * dndz
spec_zbins = glass.observations.fixed_zbins(zmin=0.9, zmax=1.8, dz=0.05)
spec_tomo_bins = glass.observations.tomo_nz_gausserr(z, spec_z, 0.0003, spec_zbins)

### Defining the tracers and angular power spectra themselves:

In [38]:
# Define an empty dictionary for the tracers:
tracers = {}

# Define the tracers:
photo_clus = {i: ccl.NumberCountsTracer(lcdm_cosmo, dndz=(z, phot_tomo_bins[i]), bias=(z, bias), mag_bias=(z, m_bias), has_rsd=True) for i in range(len(phot_tomo_bins))}
spec_clus = {i: ccl.NumberCountsTracer(lcdm_cosmo, dndz=(z, spec_tomo_bins[i]), bias=(z, bias), mag_bias=(z, m_bias), has_rsd=True) for i in range(len(spec_tomo_bins))}
wl = {i: ccl.WeakLensingTracer(lcdm_cosmo, dndz=(z, phot_tomo_bins[i]), has_shear=True) for i in range(len(phot_tomo_bins))}
cmb_l = ccl.CMBLensingTracer(lcdm_cosmo, z_source=1100.)